In [134]:
import pandas as pd
import numpy as np
import scipy.stats as st
import datetime
pd.set_option('display.max_rows', 10)

First, we import the data from the WHO growth table and our measured weights. The datasource is the [WHO](https://www.cdc.gov/growthcharts/who_charts.htm#The%20WHO%20Growth%20Charts)

In [125]:
boys_growth_table = pd.read_csv('WHO-boys-growth-table.dat',sep='\t')
weights = pd.read_csv('Elio-measured-weight.dat',sep='\t')
boys_growth_table

,Month,L,M,S,2nd (2.3rd),5th,10th,25th,50th,75th,90th,95th,98th (97.7th)
0,0,0.3487,3.3464,0.14602,2.459312,2.603994,2.757621,3.027282,3.3464,3.686659,4.011499,4.214527,4.419354
1,1,0.2297,4.4709,0.13395,3.390890,3.566165,3.752603,4.080792,4.4709,4.889123,5.290726,5.542933,5.798331
2,2,0.1970,5.5675,0.12385,4.318890,4.522344,4.738362,5.117754,5.5675,6.048448,6.509323,6.798348,7.090758
3,3,0.1738,6.3762,0.11727,5.018434,5.240269,5.475519,5.888058,6.3762,6.897306,7.395936,7.708329,8.024169
4,4,0.1553,7.0023,0.11316,5.561377,5.797135,6.046988,6.484777,7.0023,7.554286,8.082087,8.412602,8.746662
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,20,0.0087,11.3462,0.11211,9.065209,9.434095,9.826848,10.519610,11.3462,12.237130,13.098110,13.641810,14.194920
21,21,0.0029,11.5486,0.11261,9.219037,9.595435,9.996335,10.703830,11.5486,12.459830,13.341100,13.897950,14.464690
22,22,-0.0028,11.7504,0.11314,9.371554,9.755556,10.164710,10.887160,11.7504,12.682300,13.584260,14.154530,14.735200
23,23,-0.0083,11.9514,0.11369,9.522741,9.914417,10.331910,11.069460,11.9514,12.904240,13.827180,14.411080,15.005900


The table provides the statistical constants we need, but they are only defined on a monthly basis. This isn't sufficient granularity, so we'll need to interpolate the table.

First, however, we will convert the 'Month' column to datetime, starting with the birthdate

In [126]:
# convert to time series with birthdate as start

# the birthdate is Nov-1-2018
birthdate = datetime.datetime(2018,11,1)

# I'm constructing a new table, starting with Elio's birthdate
elio_growth_table = pd.DataFrame()
elio_growth_table['Date'] = pd.to_timedelta(boys_growth_table['Month'],unit='M') + birthdate

# Now, I'll copy the constants I'm interested in. The rest of the data is needed
elio_growth_table[['L','M','S']] = boys_growth_table[['L','M','S']]
elio_growth_table = elio_growth_table.set_index('Date')

# Finally, I'm going to resample the data so every day has a row
elio_growth_table = elio_growth_table.resample('1D').mean()
elio_growth_table

,L,M,S
Date,,,
2018-11-01,0.3487,3.3464,0.14602
2018-11-02,NaN,NaN,NaN
2018-11-03,NaN,NaN,NaN
2018-11-04,NaN,NaN,NaN
2018-11-05,NaN,NaN,NaN
...,...,...,...
2020-10-27,NaN,NaN,NaN
2020-10-28,NaN,NaN,NaN
2020-10-29,NaN,NaN,NaN


Now, we need to interpolate the statistical constants. The default method is linear, but I'm going to use a cubic spline to get nice a nice smooth curve

In [127]:
elio_growth_table = elio_growth_table.interpolate(method = 'cubic')
elio_growth_table

,L,M,S
Date,,,
2018-11-01,0.348700,3.346400,0.146020
2018-11-02,0.342348,3.380017,0.145614
2018-11-03,0.336193,3.414042,0.145207
2018-11-04,0.330231,3.448459,0.144799
2018-11-05,0.324460,3.483254,0.144391
...,...,...,...
2020-10-27,-0.012947,12.123712,0.114180
2020-10-28,-0.013134,12.130616,0.114200
2020-10-29,-0.013322,12.137548,0.114220


In [128]:
weights['Date'] = pd.to_datetime(weights['Date'],format='%d-%b-%Y')

# Computing Z-scores
To compute the z-score of the weight (X), we need the L, M and S parameters:

$$ Z = \frac{(X/M)^L - 1}{LS}, L \neq 0$$

To compute the weight (X), from a Z-score:

$$X = M(1 + LSZ)^{1/L}, L \neq 0$$

Z-scores can be easily translated into percentiles via the normal distribution

In [138]:
def z_score (date,weight):
    X = weight
    L, M, S = elio_growth_table.loc[date][['L','M','S']]
    Z = ((X/M)**L - 1)/(L*S)
    return Z
    
def percentile (date,weight):
    Z = z_score(date,weight)
    return st.norm.cdf(Z)

def weight_from_percentile(date,percentile):
    L, M, S = elio_growth_table.loc[date][['L','M','S']]
    Z = st.norm.ppf(percentile)
    X = M*(1 + L*S*Z)**(1/L)
    return X

In [142]:
weights['Percentile'] = weights.apply(lambda row: percentile(row['Date'],row['Weight (kg)']), axis=1)

In [149]:
# Plotting
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.io as pio
import plotly.graph_objs as go
init_notebook_mode(connected=True)